In [3]:
from pyspark.sql import SparkSession

In [4]:
spark= SparkSession.builder.appName("Regression").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/16 14:28:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/16 14:28:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
spark

In [8]:
path="smoke_detection_iot.csv"

In [9]:
training = spark.read.csv(path, header=True, inferSchema=True)

In [12]:
training.show(55)

22/09/16 14:34:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , UTC, Temperature[C], Humidity[%], TVOC[ppb], eCO2[ppm], Raw H2, Raw Ethanol, Pressure[hPa], PM1.0, PM2.5, NC0.5, NC1.0, NC2.5, CNT, Fire Alarm
 Schema: _c0, UTC, Temperature[C], Humidity[%], TVOC[ppb], eCO2[ppm], Raw H2, Raw Ethanol, Pressure[hPa], PM1.0, PM2.5, NC0.5, NC1.0, NC2.5, CNT, Fire Alarm
Expected: _c0 but found: 
CSV file: file:///Users/halef/Desktop/Coursera/PySpark/smoke_detection_iot.csv
+---+----------+--------------+-----------+---------+---------+------+-----------+-------------+-----+-----+-----+-----+-----+---+----------+
|_c0|       UTC|Temperature[C]|Humidity[%]|TVOC[ppb]|eCO2[ppm]|Raw H2|Raw Ethanol|Pressure[hPa]|PM1.0|PM2.5|NC0.5|NC1.0|NC2.5|CNT|Fire Alarm|
+---+----------+--------------+-----------+---------+---------+------+-----------+-------------+-----+-----+-----+-----+-----+---+----------+
|  0|1654733331|          20.0|      57.36|        0|      400| 12306|    

In [19]:
training.filter((training["Fire Alarm"]>0)).show(4)

22/09/16 14:40:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , UTC, Temperature[C], Humidity[%], TVOC[ppb], eCO2[ppm], Raw H2, Raw Ethanol, Pressure[hPa], PM1.0, PM2.5, NC0.5, NC1.0, NC2.5, CNT, Fire Alarm
 Schema: _c0, UTC, Temperature[C], Humidity[%], TVOC[ppb], eCO2[ppm], Raw H2, Raw Ethanol, Pressure[hPa], PM1.0, PM2.5, NC0.5, NC1.0, NC2.5, CNT, Fire Alarm
Expected: _c0 but found: 
CSV file: file:///Users/halef/Desktop/Coursera/PySpark/smoke_detection_iot.csv
+----+----------+--------------+-----------+---------+---------+------+-----------+-------------+-----+-----+-----+-----+-----+----+----------+
| _c0|       UTC|Temperature[C]|Humidity[%]|TVOC[ppb]|eCO2[ppm]|Raw H2|Raw Ethanol|Pressure[hPa]|PM1.0|PM2.5|NC0.5|NC1.0|NC2.5| CNT|Fire Alarm|
+----+----------+--------------+-----------+---------+---------+------+-----------+-------------+-----+-----+-----+-----+-----+----+----------+
|3178|1654736509|         9.381|      56.86|       11|      400| 133

In [22]:
training.select("Fire Alarm").distinct().show()

+----------+
|Fire Alarm|
+----------+
|         1|
|         0|
+----------+



In [ ]:
# As it is seen on the table that our target value (dependent variable is Fire Alarm)
# We have only 0 and 1 vales which mean that there is a fire or not.
# Let's choose the independet variables.

# ["Temperature[C]","Humidity[%]","Raw Ethanol", "Pressure[hPa]","eCO2[ppm]"]---------->>> will be our independent variables.



In [23]:
from pyspark.ml.feature import VectorAssembler


In [32]:
feature_assembler=  VectorAssembler(inputCols=["Temperature[C]","Humidity[%]","Raw Ethanol", "Pressure[hPa]","eCO2[ppm]"], outputCol="Independent Variables")

In [33]:
output = feature_assembler.transform(training)

In [34]:
output.show()

22/09/16 14:56:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , UTC, Temperature[C], Humidity[%], TVOC[ppb], eCO2[ppm], Raw H2, Raw Ethanol, Pressure[hPa], PM1.0, PM2.5, NC0.5, NC1.0, NC2.5, CNT, Fire Alarm
 Schema: _c0, UTC, Temperature[C], Humidity[%], TVOC[ppb], eCO2[ppm], Raw H2, Raw Ethanol, Pressure[hPa], PM1.0, PM2.5, NC0.5, NC1.0, NC2.5, CNT, Fire Alarm
Expected: _c0 but found: 
CSV file: file:///Users/halef/Desktop/Coursera/PySpark/smoke_detection_iot.csv
+---+----------+--------------+-----------+---------+---------+------+-----------+-------------+-----+-----+-----+-----+-----+---+----------+---------------------+
|_c0|       UTC|Temperature[C]|Humidity[%]|TVOC[ppb]|eCO2[ppm]|Raw H2|Raw Ethanol|Pressure[hPa]|PM1.0|PM2.5|NC0.5|NC1.0|NC2.5|CNT|Fire Alarm|Independent Variables|
+---+----------+--------------+-----------+---------+---------+------+-----------+-------------+-----+-----+-----+-----+-----+---+----------+---------------------+
|  0|165

In [35]:
output.columns

['_c0',
 'UTC',
 'Temperature[C]',
 'Humidity[%]',
 'TVOC[ppb]',
 'eCO2[ppm]',
 'Raw H2',
 'Raw Ethanol',
 'Pressure[hPa]',
 'PM1.0',
 'PM2.5',
 'NC0.5',
 'NC1.0',
 'NC2.5',
 'CNT',
 'Fire Alarm',
 'Independent Variables']

In [38]:
# The IndependentVariables are shown there adn let get finalize the data.

final_data = output.select("Independent Variables","Fire Alarm")



In [41]:
final_data.show(5)

+---------------------+----------+
|Independent Variables|Fire Alarm|
+---------------------+----------+
| [20.0,57.36,18520...|         0|
| [20.015,56.67,186...|         0|
| [20.029,55.96,187...|         0|
| [20.044,55.28,188...|         0|
| [20.059,54.69,189...|         0|
+---------------------+----------+
only showing top 5 rows



In [43]:
from pyspark.ml.regression import LinearRegression

In [47]:
train_data, test_data = final_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol= "Independent Variables", labelCol="Fire Alarm")
regressor=regressor.fit(train_data)

22/09/16 15:01:54 WARN Instrumentation: [4ca28167] regParam is zero, which might cause numerical instability and overfitting.


In [48]:
regressor.coefficients

DenseVector([-0.0033, 0.0244, -0.0004, -0.0764, -0.0001])

In [49]:
regressor.intercept

78.92938678886158

In [50]:
prediction_results = regressor.evaluate(test_data)

In [53]:
prediction_results.predictions.show(5)

+---------------------+----------+------------------+
|Independent Variables|Fire Alarm|        prediction|
+---------------------+----------+------------------+
| [-22.01,47.99,193...|         1| 0.974931344472509|
| [-22.01,48.02,193...|         1|0.9779545556699532|
| [-22.01,48.25,193...|         1|0.9797398610176202|
| [-22.01,49.24,193...|         1|1.0100389475031477|
| [-22.009,51.21,19...|         1|1.0577128204452464|
+---------------------+----------+------------------+
only showing top 5 rows



In [56]:
prediction_results.meanAbsoluteError 


0.280494792178067

It is easily seen that mae values is close to 0 and we can say that our model is highly accurate.

In [57]:
prediction_results.meanSquaredError

0.13143417141901664

It is easily seen that mean square error error is close to 0 and we can say that our model is more accurate.
We can state that our model is perfect estimator.